<a href="https://colab.research.google.com/github/t8101349/Costco-discounted-goods-crawler-bot/blob/main/costco_linebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk flask


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.9/779.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.2 MB/s eta 0:00:00


In [67]:
%%writefile linebot.py
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import os
import requests
import json
import time

app = Flask(__name__)

CHANNEL_ACCESS_TOKEN = 'ik9reNYdHWJhkqdey1cIrtKNF/yMKWyCGTZmkfeZH4FtGSj/hwZmHputQN/95c5dW29MRScjKcLwRFwD67vw8pMON62xtn/lbxm/7X7jVHplqvOtFM9jAGb+azrpqSEULEzdr4OSCJj2aAYm2wH21AdB04t89/1O/w1cDnyilFU='
CHANNEL_SECRET = 'a32b70f15b6fe55ec4ae766e1325cd16'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# Costco 折扣抓取邏輯
api_url = "https://www.costco.com.tw/rest/v2/taiwan/products/search"
params = {
    "fields": "FULL",
    "query": ":BazaarVoiceRating-desc:bazaarVoiceAverageRatingFacet:4-5:bazaarVoiceAverageRatingFacet:5:bazaarVoiceAverageRatingFacet:3-4",
    "pageSize": 3,
    "sort": "BazaarVoiceRating-desc",
    "category": "Coupon",
    "lang": "zh_TW",
    "curr": "TWD",
    "currentPage": 0
}

def fetch_costco_data():
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json"}
    response = requests.get(api_url, headers=headers, params=params)
    products_info = []

    if response.status_code == 200:
        products = response.json().get('products', [])
        for product in products:
            name = product.get("name")
            price = product.get("price", {'value':"售完"}).get("value")
            currency = product.get("price", {}).get("currencyIso", "")
            rating = product.get("averageRating", "無評價")
            image_url = 'https://www.costco.com.tw' + product.get("images", [{}])[0].get("url", "無圖片")
            products_info.append(f"商品: {name}\n價格: {price} {currency}\n評價: {rating}\n{image_url}\n")
    return products_info

@app.route('/callback', methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature')
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    if user_message == "查詢折扣":
        costco_data = fetch_costco_data()
        reply = "\n".join(costco_data) if costco_data else "暫無優惠資訊"
    else:
        reply = f"你說了：{user_message}"
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))

Overwriting linebot.py


In [ ]:
!apt-get update
!apt-get install -y curl
!curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-430.0.0-linux-x86_64.tar.gz
!tar -xzf google-cloud-sdk-430.0.0-linux-x86_64.tar.gz
!./google-cloud-sdk/install.sh -q
!./google-cloud-sdk/bin/gcloud init

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [27]:
# 驗證 GCP Service Account
from google.colab import files
uploaded = files.upload()  # 上傳 service-account-key.json


Saving costco-line-e1c5befd361d.json to costco-line-e1c5befd361d (2).json


In [25]:
!gcloud config set project costco-line


Updated property [core/project].


In [26]:
!./google-cloud-sdk/bin/gcloud auth activate-service-account --key-file=/content/costco-line-e1c5befd361d.json

# 設定專案與區域
!./google-cloud-sdk/bin/gcloud config set project costco-line
!./google-cloud-sdk/bin/gcloud config set run/region asia-east1

Activated service account credentials for: [costco-line@costco-line.iam.gserviceaccount.com]
Updated property [core/project].
Updated property [run/region].


In [68]:
%%writefile Dockerfile
FROM python:3.10-slim
WORKDIR /app
COPY . .
RUN python3 -m venv .venv
ENV PATH="/app/.venv/bin:$PATH"
RUN pip install --upgrade pip
RUN pip install --no-cache-dir flask line-bot-sdk
ENV PORT 8080
CMD ["python", "linebot.py"]

Overwriting Dockerfile


In [50]:
!gsutil mb -l asia-east1 -p costco-line gs://costco-bot

Creating gs://costco-bot/...


In [69]:
# 建立映像並推送到 Cloud Run
#!./google-cloud-sdk/bin/gcloud builds submit --tag gcr.io/costco-line/line-bot
!./google-cloud-sdk/bin/gcloud builds submit --tag gcr.io/costco-line/line-bot --gcs-log-dir=gs://costco-bot


Creating temporary tarball archive of 24284 file(s) totalling 984.0 MiB before compression.
Uploading tarball of [.] to [gs://costco-line_cloudbuild/source/1743584080.562507-e28641957f304dd7b37c44e509352014.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/costco-line/locations/global/builds/19724d1c-b647-484b-8c3d-af6cfde28738].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/19724d1c-b647-484b-8c3d-af6cfde28738?project=498723905289 ].
 REMOTE BUILD OUTPUT
starting build "19724d1c-b647-484b-8c3d-af6cfde28738"

FETCHSOURCE
Fetching storage object: gs://costco-line_cloudbuild/source/1743584080.562507-e28641957f304dd7b37c44e509352014.tgz#1743584262660303
Copying gs://costco-line_cloudbuild/source/1743584080.562507-e28641957f304dd7b37c44e509352014.tgz#1743584262660303...
| [1 files][379.4 MiB/379.4 MiB]                                                
Operation completed over 1 objects/379.4 MiB.
tar: .config/gce: time stamp 2040-01-01 00:00:00 is 4654

In [71]:
!./google-cloud-sdk/bin/gcloud run deploy line-bot --image gcs-log-dir=gs://costco-bot/line-bot --platform managed --allow-unauthenticated



Deploying container to Cloud Run service [line-bot] in project [costco-line] region [asia-east1]
ERROR: (gcloud.run.deploy) spec.template.spec.containers[0].image: Expected an image path like [host/]repo-path[:tag and/or @digest], where host is one of [region.]gcr.io, [region-]docker.pkg.dev or docker.io but obtained gcs-log-dir=gs://costco-bot/line-bot. To deploy container images from other public or private registries, set up an Artifact Registry remote repository. See https://cloud.google.com/artifact-registry/docs/repositories/remote-repo.
